# Problem Set 2: Intro to Pandas

## Part 1

In this problem set we are going to be working with a processed O-D dataset, Pandas, and Matplotlib.

First, a short example will introduce us to the dataset, some data wrangling functions of Pandas, and a few plots with Matplotlib. 
The OD data was collected based on cell phone towers. Usually, OD datasets are quite big, and contain individual data points, with origin-destination information between cell phone towers. In this case, the dataset has previously been aggregated and processed. While the text files are still very big, their size is significantly smaller than the original dataset. 
The files that follow the "Rn_OD_X-Y" naming structure contain the origin-destination flow from hour X to hour Y with the following columns:

* Origin tower ID
* Destination tower ID
* Closest road intersection to origin 
* Closest intersection to destination
* Origin-destination flow

The file "towers_index.txt" contains the locations of all the cell towers with the following columns. The row number is the id of the cell tower starting with 1.

* Latitude
* Longitude
* Node ID of the closest intersection to the tower

Throughout the exercises we will be continuously using Pandas and some of its merging functionality to join the numerous OD files available. We will be building exploratory data visualizations, finding urban patterns and locating divergent areas within the dataset. Some useful documentation for merging data with Pandas can be found at:


http://pandas.pydata.org/pandas-docs/stable/merging.html

First, lets import some of our libraries. If you haven't installed seaborn already, please type on your command line:

```Python
pip install seaborn
```

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt

# This line lets us plot on our ipython notebook
%matplotlib inline

Let’s read one of the OD files with Pandas. Since the file has no header, we need to provide the header names. Luckily, we have all the information we need on the `DATA_STRUCTURES.txt` file. 

In [ ]:
df_01=pd.read_csv("data/Hourly Flows/Rn_OD_0-1.txt", header=None, 
               names=['oid', 'did', 'close_rid_origin', 'close_rid_dest', 'flow'])
df_01.head()

We need to import every file with a separate function. 

`Hint: You can import multiple files with a for loop, or building a function, without having to declare them individually`

In [ ]:
df_12=pd.read_csv("data/Hourly Flows/Rn_OD_1-2.txt", header=None, 
               names=['oid', 'did', 'close_rid_origin', 'close_rid_dest', 'flow'])
df_12.head()

If you notice, the df structure of both files is similar. We can perform a SQL style join based on a column or index on either of the df’s. In this case, since the rest of the columns are the same, we are only keeping the flow column of the second df.

`Hint: You can merge multiple files with a for loop, or building a function, without having to do so individually`

In [ ]:
combined_flow = df_01.join(df_12['flow'], lsuffix='0-1', rsuffix='1-2')
combined_flow[:10]

We can visualize our newly merged tables with a basic area plot. This plot visualizes the flow within the cell phone towers throughout the 2 different time ranges. By slicing the dataset, we are only plotting some of the towers. 

In [ ]:
combined_flow[:5][['flow0-1', 'flow1-2']].transpose().plot(kind='area', stacked=False, 
                                                            legend=True, title='Tower Flows')



**Based on the code snippets, and your previous code your job is to build:**

* A function that merges the 24 hour ranges into a single DataFrame. The function must have the following inputs/outputs:
    1. Inputs: 
      1. A file path to extract the different files from
      * A column name to merge into a single DataFrame
      * A list of column names for the DataFrame
    2. Outputs: 
      1. A DataFrame with the merged columns

In [ ]:
# Add your code below:
#create a list of file names to import
filenames = []
for x in range(0,24):
    string = "data/Hourly Flows/Rn_OD_"+str(x)+"-"+str(x+1)+".txt"
    filenames.append(string);
    
#create a list of dataframes built from the files
frames = []
for x in range(0,24):
    df = pd.read_csv(filenames[x], header=None, names=['oid', 'did', 'close_rid_origin', 'close_rid_dest', 'flow'])
    frames.append(df);

    
#recursively merge all the dataframes, matching on unique oid+did combinations
all_flow = df_01
for x in range(1,24):
    all_flow = pd.merge(all_flow, frames[x], how='outer', on=['oid','did','close_rid_origin','close_rid_dest'],
                        suffixes=('','_'+str(x)+str(x+1)));

all_flow.rename(columns={'flow': 'flow_01'}, inplace=True)

#check to make sure it looks right
all_flow[:10]




* Using all the towers: Make an area plot that shows the relationship between OD flows throughout the 24 hr ranges.

In [ ]:
# Add your code below:
#generate the necessary column names
col_names=[]
for x in range(0,24):
    string='flow_'+str(x)+str(x+1)
    col_names.append(string);

    
#make the chart
#only the first 10 oid+did combinations are shown; displaying all 477,893 rows would be unwieldy
all_flow[:10][col_names].transpose().plot(kind='area', stacked=False, 
                                                            legend=True, title='Ten Inter-Tower Flows')

# if time: fix x-labels to show actual times of day

In [ ]:
#make a dataframe that groups by oid, sums the flows, and sorts by most active towers
group_flow = all_flow.groupby('oid').sum()
group_flow.reset_index(level=0, inplace=True)
group_flow['sum']=group_flow[col_names].sum(axis=1)
group_flow_sorted = group_flow.sort_values(by='sum', ascending=False)

#now plot the top 10
group_flow_sorted[:10][col_names].transpose().plot(kind='area', stacked=False, legend=True,
                                                title='Hourly Flows for 10 Most Active OIDs')


** Choose ONE of the following TWO questions.**
* An area plot with **four** different areas:
    1. The average flow values across **all** towers
    2. The maximum flow values across **all** towers
    3. The minimum flow values across **all** towers
    4. The median flow values across **all** towers

In [ ]:
# Add your code below:
#make a table of summary statistics and clean it up
flow_stats = all_flow.describe()
flow_stats.drop(['count','std','25%','75%'], inplace=True)
flow_stats.drop(['oid','did','close_rid_origin','close_rid_dest'], axis=1, inplace=True)
#flow_stats

#plot it
flow_stats[:][col_names].transpose().plot(kind='area', stacked=False, 
                                                            legend=True, title='Summary Stats for All Tower Flows')
#Can't see detail, so remove the outlier
flow_stats.drop('max', inplace=True)

flow_stats[:][col_names].transpose().plot(kind='area', stacked=False, 
                                                            legend=True, title='Zoomed in:')


In [ ]:
# Repeat the above, but now using the top 10 list generated earlier

#make a table of summary statistics and clean it up
group_stats = group_flow_sorted[:10].describe()
# group_stats = group_flow_sorted.describe()
group_stats.drop(['count','std','25%','75%'], inplace=True)
group_stats.drop(['oid','did','close_rid_origin','close_rid_dest'], axis=1, inplace=True)
#flow_stats


#plot it
group_stats[:][col_names].transpose().plot(kind='area', stacked=False, 
                                                            legend=True, title='Summary Stats for 10 Most Active OIDs')




* Based on the overall trends that you just noticed with the previous plots, construct:
    1. An area plot based on the values of 5 towers with flow values that are closer to the average trend.
    2. An area plot based on the outlier values. You will plot 3 towers with flow values that are closer to the minimum trend, and towers with flow values that are closer to the maximum trend.

In [ ]:
# Add your code below:
# Chose the other question

## Part 2

Now we will use the additional file provided to display the geographical location of every tower. We will first import the file and look at its structure. Since the file has no header, we will use the information provided on the `DATA_STRUCTURES.txt` file. The row number corresponds to the tower index; in order to relate every tower with the OD files, we will add an additional column with the row number as an index. We can do so with the following `DataFrame` function:
```Python
DataFrame.reset_index(level=0, inplace=True)
```


In [ ]:
towers=pd.read_csv("data/Hourly Flows/towers_index.txt", header=None, 
               names=['Latitude', 'Longitude', 'Closest node ID'])
# Use the row value as the index column
towers.reset_index(level=0, inplace=True)
towers.head()

If you look at the structure of the OD file, every tower is repeated several times: for every origin tower, there are several destinations. In SQL terms, we can use a `one-to-many` relationship to assign the geographic values of the towers, to the origin or destination towers. By joining a `DataFrame` `on` a given column, Pandas will assign the row value of every element that matches the value of the given column on the other `DataFrame`. We will also group the origin values of the OD Dataframe, and get their average before joining it with the towers’ geographical location. 

In [ ]:
# from the OD df, use only two columns, then group them by 'oid'
grouped_01 = df_01[['flow','oid']].groupby('oid').mean()

# use the oid as a column, not the index
grouped_01.reset_index(level=0, inplace=True)

# join with the tower df based on the oid
OD_location = grouped_01.join(towers, on='oid')
OD_location.head()

Now that we have a new `DataFrame` with an average flow value for every origin tower, and their corresponding geographical location, we can use their lat/lon values to create a scatter plot. We can even vary their size based on the average flow. Since the flow variation across tower is not that big, we can use a log function to better visualize it on our plot. Numpy has a quick log implementation:
```Python
np.log(value)
```


In [ ]:
plt.scatter(OD_location['Longitude'],OD_location['Latitude'], s=np.log(OD_location['flow'])*100)
plt.show()

**Based on the code snippets, and your previous code your job is to build:**

* A function that creates a [color range](http://matplotlib.org/users/colormaps.html) based on a `numpy array` of numeric values [(Example 1)](http://stackoverflow.com/questions/12965075/matplotlib-scatter-plot-colour-as-function-of-third-variable), [(Example 2)](http://stackoverflow.com/questions/11364745/how-can-i-turn-a-numpy-array-into-a-matplotlib-colormap). The function must have the following inputs/outputs:
    1. Inputs: 
      1. A `numpy` `array` or a `Pandas DataSeries`
      * Two different colors to interpolate values between.
    2. Outputs: 
      1. A `numpy array` of `colors` or `color values` 
      

In [ ]:
# Add your code below:

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.colors import LinearSegmentedColormap
%matplotlib inline


# make a function that transforms two colors into a colormap

def make_cmap(ColorP, ColorQ):
    cmdict = {'red':  [(0.0,  0.0, 0.0),
                       (1.0,  0.0, 0.0)],

             'green': [(0.0,  0.0, 0.0),
                       (1.0,  0.0, 0.0)],

             'blue':  [(0.0,  0.0, 0.0),
                        (1.0,  0.0, 0.0)]}

#Convert input colors to rgb and feed the values into the color matrix
    colorA = colors.colorConverter.to_rgb(ColorP)
    colorB = colors.colorConverter.to_rgb(ColorQ)

    cmdict['red'] = [(0.0, colorA[0], colorA[0]),
                     (1.0, colorB[0], colorB[0])]
    cmdict['green'] = [(0.0, colorA[1], colorA[1]),
                       (1.0, colorB[1], colorB[1])]
    cmdict['blue'] = [(0.0, colorA[2], colorA[2]),
                      (1.0, colorB[2], colorB[2])]

    my_cmap = LinearSegmentedColormap('MyCMAP', cmdict)
    return my_cmap;

# try it on a sample dataframe
x = OD_location['Longitude']
y = OD_location['Latitude']
z = OD_location['index']


plt.scatter(x, y, marker='+', c=z, cmap=make_cmap('aqua','orange'), s=150, linewidths=1)
plt.show()


# The following function, written to produce a list of colors from the colormap, doesn't work.
# However it is easy enough to pass both a colormap and an array into scatter()
# so that is how I generated the later scatterplots.

# def pick_colors(colorP, colorQ, z):
#     cmtemp = make_cmap(colorP, colorQ)
#     cmtemp2 = plt.cm.get_cmap(cmtemp)
#     z_array = np.array(z)
#     z_norm = plt.Normalize(vmin=0,vmax=1)
#     color_list = []
#     for item in z_array:
#         colortemp = cmtemp2((item-z_array.min())/z_array.ptp())
#         color_list.append(colortemp);
#     return color_list;


    

### All the plots below should be scatter plots. Parameterize the size of the points to the obtained values.

* A plot that visualizes the average flow values using the 24 hour ranges. Color the plots using the function to create the color array.
    * The `Dataframe` must be grouped and averaged by 'oid'
    * The flow values must be averaged across the 24 hr ranges. IE
    ```Python
    flow_ranges = [5, 5, 5, 5, 5, 5, 10, 10, 10, 10, 10, 10]
    avg_flow = 7.5
    ```

In [ ]:
# Add your code below:

#add a column to average across the timeframe
all_flow['avg_flow'] = all_flow[col_names].mean(axis=1)

# group them on OID, averaging values within each
grouped_avg = all_flow[['oid','avg_flow']].groupby('oid').mean()
grouped_avg.reset_index(level=0, inplace=True)

# merge it with the towers data, first forcing tower id to start at "1" so it aligns properly
towers['oid'] = towers['index']+1
flow_loc_avg = pd.merge(grouped_avg, towers, how='outer', on='oid', suffixes=('',''))

# define z and use the plotting function
z_avg = flow_loc_avg['avg_flow']

plt.scatter(flow_loc_avg['Longitude'], flow_loc_avg['Latitude'], np.log(z_avg)*100,
           c=z_avg, cmap=make_cmap('white','red'))
plt.ylabel('Latitude')
plt.xlabel('Longitude')
plt.title('Average Hourly Flows by Location')
plt.show()


* A plot that visualizes the maximum flow values using the 24 hour ranges. Color the plots using the function to create the color array.
    * The `Dataframe` must be grouped by 'oid', you can then use these values to find the `max()`
    * Use those flow values to find the `max()` across the 24 hr ranges. IE
    ```Python
    flow_ranges = [5, 5, 5, 5, 5, 5, 10, 10, 10, 10, 10, 100]
    max_flow = 100
    ```

In [ ]:
# Add your code below:
#add the max flow column
all_flow['max_flow'] = all_flow[col_names].max(axis=1)

# group on oid, averaging the maxes within each
grouped_max = all_flow[['oid','max_flow']].groupby('oid').mean()
grouped_max.reset_index(level=0, inplace=True)

#merge it with the towers data
flow_loc_max = pd.merge(grouped_max, towers, how='outer', on='oid', suffixes=('',''))

#define z and use the plotting function
z_max = flow_loc_max['max_flow']

plt.scatter(flow_loc_max['Longitude'], flow_loc_max['Latitude'], np.log(z_max)*100,
           c=z_max, cmap=make_cmap('white','blue'))
plt.ylabel('Latitude')
plt.xlabel('Longitude')
plt.title('Maximum Daily Flows by Location')
plt.show()


* A plot that visualizes the minimum flow values using the 24 hour ranges. Color the plots using the function to create the color array.
    * The `Dataframe` must be grouped by 'oid', you can then use these values to find the `min()`
    * Use those flow values to find the `min()` across the 24 hr ranges. IE
    ```Python
    flow_ranges = [5, 5, 5, 5, 5, 5, 10, 10, 10, 10, 10, 100]
    min_flow = 5
    ```

In [ ]:
# Add your code below:
#add the min flow column
all_flow['min_flow'] = all_flow[col_names].min(axis=1)

# group on oid, averaging the mins within each
grouped_min = all_flow[['oid','min_flow']].groupby('oid').mean()
grouped_min.reset_index(level=0, inplace=True)

#merge it with the towers data
flow_loc_min = pd.merge(grouped_min, towers, how='outer', on='oid', suffixes=('',''))

#define z and use the plotting function
z_min = flow_loc_min['min_flow']

plt.scatter(flow_loc_min['Longitude'], flow_loc_min['Latitude'], np.log(z_min)*100,
           c=z_min, cmap=make_cmap('white','green'))
plt.ylabel('Latitude')
plt.xlabel('Longitude')
plt.title('Minimum Daily Flows by Location')
plt.show()


## Graphic Presentation
#### Make sure to label all the axis, add legends and units (where appropriate).


## Code Quality
#### While code performance and optimization won't count, all the code should be highly readable, and reusable. Where possible, create functions, build helper functions where needed, and make sure the code is self-explanatory.